# LOFO (Leave One Feature Out)

## Lofo is a method for estimating the importance of a feature.

## Advantage
- It is model agnostic. You won't need different importance types using for different models. (weights for Logistic Regression, gain for GBDT etc.)
- It is robust against granular features compared to split importance.
- Other importance types ignore validation schemes. LOFO depends on the validation scheme. This will allow us to see that Census_OSVersion is not important given that the fold splits are according to time.

# Referece

- https://www.kaggle.com/c/microsoft-malware-prediction/discussion/79415
- https://github.com/aerdem4/lofo-importance/issues/40

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install lofo-importance

In [ ]:
import pandas as pd
import datatable as dt
from sklearn.model_selection import KFold
from lofo import LOFOImportance, Dataset, plot_importance

In [ ]:
train_df = dt.fread("../input/tabular-playground-series-nov-2021/train.csv").to_pandas()

In [ ]:
train_df

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
 train_df = reduce_memory_usage(train_df)

In [ ]:
sample_df = train_df.sample(frac=0.01, random_state=32)
# sample_df.sort_values("f0", inplace=True)

cv = KFold(n_splits=5, shuffle=True, random_state=32)
target = "target"
features = [col for col in train_df.columns if col != target]

In [ ]:
dataset = Dataset(sample_df, target=target, features=features)

In [ ]:
lofo = LOFOImportance(dataset, cv=cv, scoring="roc_auc")
importance_df = lofo.get_importance()

In [ ]:
importance_df

In [ ]:
plot_importance(importance_df, figsize=(12, 20))